In [13]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
from datetime import datetime


In [14]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
}

def get_request_with_retries(url, headers, retries=3, timeout=20):
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            if response.status_code == 200:
                return response
        except Exception as e:
            print(f"⚠️ Intento {i+1} fallido para {url}: {e}")
            time.sleep(2)  # Esperar 2 segundos antes de reintentar
    return None

In [15]:
# Lista de secciones
secciones = ['ocio', 'viajes', 'shopping', 'educacion', 'salud', 'estilo-de-vida']

articulos = []

for seccion in secciones:
    print(f"\n🔎 Visitando sección: {seccion.upper()}")

    for pagina in range(1, 7):  # 6 primeras páginas (de 1 a 6)
        if pagina == 1:
            url = f'https://quehacerconlosninos.es/{seccion}/'
        else:
            url = f'https://quehacerconlosninos.es/{seccion}/page/{pagina}/'

        try:
            response = requests.get(url, headers=headers, timeout=10)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                titulos = soup.find_all('h3', class_='elementor-post__title')

                for t in titulos:
                    titulo = t.get_text(strip=True)
                    link = t.find('a')['href']

                    try:
                        # Entrar al artículo
                        response_art = get_request_with_retries(link, headers)
                        soup_art = BeautifulSoup(response_art.text, 'html.parser')
                        article = soup_art.find('div', class_='elementor-widget-theme-post-content')

                        if article:
                            parrafos = article.find_all('p')
                            texto = " ".join(p.get_text(strip=True) for p in parrafos)

                            if texto.strip():
                                articulos.append({
                                    'seccion': seccion,
                                    'titulo': titulo,
                                    'url': link,
                                    'contenido': texto
                                })
                                print(f"✅ Extraído: {titulo}")
                            else:
                                print(f"⚠️ Contenido vacío: {titulo}")
                        else:
                            print(f"⚠️ No se encontró contenido en: {link}")

                    except Exception as e:
                        print(f"❌ Error en el artículo {link}: {e}")

                    time.sleep(1.0)  # Pausa entre artículos

            else:
                print(f'❌ No se pudo acceder a {url}')

        except Exception as e:
            print(f"❌ Error accediendo a {url}: {e}")

# Crear DataFrame
df = pd.DataFrame(articulos)

# (Opcional) Guardarlo
# df.to_csv('articulos_por_seccion_y_paginas.csv', index=False, encoding='utf-8')

# print("\n🎯 Scrap terminado. Artículos extraídos:", len(df))



🔎 Visitando sección: OCIO
❌ Error accediendo a https://quehacerconlosninos.es/ocio/: HTTPSConnectionPool(host='quehacerconlosninos.es', port=443): Read timed out. (read timeout=10)


KeyboardInterrupt: 

In [5]:
df = pd.DataFrame(articulos)

In [8]:
df.to_csv('Articulos2.csv',index=False)

In [32]:
pd.set_option('display.max_colwidth', None)

In [2]:
regex_fechas = r"(?:\b(?:lunes|martes|miércoles|jueves|viernes|sábado|domingo)\s*)?\b\d{1,2}\s+de\s+(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)(?:\s+de\s+\d{4})?"
regex_precios = r"\d+(?:[\.,]\d+)? ?€"
meses = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}

def extraer_fechas_separadas(texto):
    fechas_encontradas = re.findall(regex_fechas, texto, flags=re.IGNORECASE)

    if not fechas_encontradas:
        return None, None
    elif len(fechas_encontradas) == 1:
        return fechas_encontradas[0], fechas_encontradas[0]  # Solo una fecha, inicio = fin
    else:
        return fechas_encontradas[0], fechas_encontradas[1]  # Inicio = primera fecha, fin = segunda fecha

def extraer_precio(texto):
    if pd.isnull(texto):
        return "No especificado"
    
    texto = texto.lower()
    
    # Buscar si contiene la palabra "gratis"
    if "gratis" in texto:
        return "Gratis"
    
    # Buscar precios en euros como "15€", "6 €", "20 €", etc.
    precios = re.findall(regex_precios, texto)
    if precios:
        return precios[0].replace(" ", "")
    
    return "No especificado"

def corregir_texto(texto):
    # Insertar espacio entre letras y números
    texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)
    texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
    return texto

def extraer_fecha_contexto_basico(texto):
    if pd.isnull(texto):
        return None

    texto = texto.lower()
    patrones = re.findall(r"(\d{1,2}) de (\w+)(?: de (\d{4}))?", texto)

    if not patrones:
        return None

    for dia, mes_nombre, año in patrones:
        mes = meses.get(mes_nombre)
        if mes is None:
            continue  # Ignorar si el mes no es reconocido

        año = int(año) if año else datetime.today().year

        try:
            fecha = datetime(int(año), int(mes), int(dia))
            hoy = datetime.today()
            return {
                "fecha_normalizada": fecha.strftime("%Y-%m-%d"),
                "dia_semana": fecha.strftime("%A"),
                "mes": fecha.strftime("%B"),
                "año": fecha.year,
                "es_este_mes": fecha.month == hoy.month and fecha.year == hoy.year,
                "es_este_fin_de_semana": fecha.weekday() in [5, 6] and 0 <= (fecha - hoy).days <= 7
            }
        except ValueError:
            continue

    return None


In [3]:
df = pd.read_csv('..\data\processed\Articulos_LLM1.csv')

In [4]:
df['contenido'] = df['contenido'].apply(corregir_texto)
df[['fecha_inicio', 'fecha_fin']] = df['contenido'].apply(
    lambda x: pd.Series(extraer_fechas_separadas(x))
)
df['fechas'] = df['contenido'].apply(extraer_fechas_separadas)
df['Precios'] = df['contenido'].apply(extraer_precio)

In [5]:
df.head()

,titulo,url,contenido,fechas_contexto,fechas,Precios,fecha_inicio,fecha_fin
0,"San Isidro 2025 en Madrid: tradiciones, verben...",https://quehacerconlosninos.es/san-isidro-2025...,San Isidro 2025 en Madridya tiene fecha: del 3...,"{'fecha_normalizada': '2025-05-15', 'dia_seman...","(15 de mayo, 15 de mayo)",No especificado,15 de mayo,15 de mayo
1,"Feria del Barro de Aranjuez 2025: talleres, co...",https://quehacerconlosninos.es/feria_del_barro...,Laferia del Barro de Aranjuezregresa este fin ...,"{'fecha_normalizada': '2025-04-27', 'dia_seman...","(domingo 27 de abril, domingo 27 de abril)",No especificado,domingo 27 de abril,domingo 27 de abril
2,The Champions Burger en Las Rozas: hamburguesa...,https://quehacerconlosninos.es/the-champions_b...,Desde este mismo momento y hasta el domingo 11...,"{'fecha_normalizada': '2025-05-11', 'dia_seman...","(domingo 11 de mayo, domingo 11 de mayo)",No especificado,domingo 11 de mayo,domingo 11 de mayo
3,Rutas de senderismo en Madrid con bebés: plane...,https://quehacerconlosninos.es/rutas-de-sender...,Hacer rutas de senderismo en Madrid con bebés ...,NaN,"(None, None)",No especificado,None,None
4,Programa completo San Isidro 2025 en Madrid,https://quehacerconlosninos.es/programa-comple...,Madrid ya calienta motores para celebrar una d...,"{'fecha_normalizada': '2025-05-15', 'dia_seman...","(15 de mayo, 30 de abril)",No especificado,15 de mayo,30 de abril


In [6]:
df.to_csv('Articulos_LLM2.csv',index=False)